<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/%EC%88%98%EC%B9%98%EB%AF%B8%EB%B6%84%EC%9D%84_%ED%86%B5%ED%95%9C_%ED%95%99%EC%8A%B5_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pylab as plt

#Gradient, softmax, corss_entorpy_error, sigmoid 구현

In [ ]:
def _numerical_gradient_no_batch(f, x): # 수치 gradient, 앞의 수치미분과 같은 개념
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성(이 경우 x와 같은 차원의 제로 벡터)

    # gradient는 각 축에 대한 방향 미분을 모아놓은 벡터이므로 각 방향에 대한 수치미분을 grad 벡터에 차례로 대입하는 과정이다.
    for idx in range(x.size):
        tmp_val = x[idx]

        # f(x+h) 계산
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)

        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] =  (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 값 복원

    return grad

In [ ]:
def numerical_gradient(f, X): # 배치처리를 한 경우(행렬, 텐서 미분)
    if X.ndim == 1: # 1차원, 그러니까 행렬로 묶이지 않은 벡터인 경우
        return _numerical_gradient_no_batch(f, X)
    else:
        grad = np.zeros_like(X)

        for idx, x in enumerate(X): # enumerate 설명1, 행벡터를 차례로 불러와 미분한다.
            grad[idx] = _numerical_gradient_no_batch(f, x)

        return grad

In [ ]:
def cross_entropy_error(y, t):
    if y.ndim == 1: # y가 벡터인 경우, 배치처리를 안 한 경우
        t = t.reshape(1, t.size) # 10차원 벡터를 행렬로 변환.(개념상.)
        y = y.reshape(1, y.size)

    if t.size == y.size: # 라벨이 원-핫 인코딩이 돼어 있다면, 그러니까 입력 값이 10차원 벡터라면, size함수는 메트릭스의 원소갯수를 반환한다.
        t = t.argmax(axis=1) # 원-핫 인코딩 이전으로 되돌리기 위해 작성.

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size # 설명1, 확률벡터로 이루어진 행렬에서 알맞은 인덱스를 얻기위한 과정.

In [ ]:
def softmax(x):
    if x.ndim == 2:   # x가 행렬일 경우, 즉 배치처리를 해서 각 행이 확률벡터인 메트릭스를 입력받은 경우.
        x = x.T # 각 행이 확률벡터인 초기 메트릭스에서 각 열이 확률벡터인 메트릭스로 전치 시킨다.
        x = x - np.max(x, axis = 0) # np.max(x, axis = 0)는 각 열에 대한 최댓값을 저장한 리스트(벡터)를 반환 한다. 그리고 x에 대한 - 연산은 각 열의 확률벡터에 각 열의 최댓값을 빼주는 연산이다.(밑의 오버플로우 방지를 위한 최댓값 빼기와 같음.)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0) # 위와 같은 행렬과 벡터의 연산 원리.( 밑의 예제 참고.)
        return y.T

    x = x - np.max(x) # 오버플로를 방지하기위해 작성. 노트참고. , vector에 scalar를 빼면 numpy에서 알아서 원소별로 연산을 한다.
    return np.exp(x) / np.sum(np.exp(x))  # normalize

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  # np.exp(x)는 밑이 자연상수 e 이고 x가 지수인 수를 반환한다.

# 1

In [ ]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01): # std는 표준편차이고 특별히 지정하지 않으면 0.01로 지정한다.

        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) # 표준정규분포(평균이 0이고 표준편차가 1)에 상수 a를 곱하면 평균이 0이고 표준편차가 a인 정규분포를 생성한다.
        self.params['b1'] = np.zeros(hidden_size) # bias는 0으로 지정한다.
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y

    def loss(self, x, t):
        y = self.predict(x)

        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, aixs = 1)
        t = np.argmax(t, axis = 1)

        accuracy = np.sum(y == t) / float(x.shape[0])

        return accuracy

    def numerical_gradient  8/20